# Лабораторная работа №5
## $1 | \text{s-batch} | \sum{w_ic_i}$

**Задача**: собрать работы в группы для обработки на одной машине так, чтобы
минимизировать взвешенную сумму окончания всех работ. В каждой группе
время окончания работ равно времени окончания последней работы в группе. 
Длительность выполнения всей группы работ равна сумме длительностей
работ. При переходе от одной группы к другой машина требует переналадки
$\tau$ (простой.) 

Реализовать алгоритм решения с помощью библиотеки Pyomo и собственный жадный алгоритм для решения данной задачи, создать универсальный формат задания задачи.

In [112]:
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import networkx as nx
from functools import reduce
import pandas as pd
import shutil
import sys
import json
import os.path
from pyomo.environ import *
from pyomo.gdp import *
from pyomo.contrib.latex_printer import latex_printer
import script

In [113]:
# Запись задачи в JSON формат
def WriteJson(dictionary: dict, name = "task.json"):
    with open(name, 'w') as f:
        json.dump(dictionary, f)

# Чтение задачи из JSON формата
def ReadJson(name = "task.json") -> dict:
    with open(name, 'r') as f:
        m = json.loads(f.read())
        return m

In [114]:
# Отрисовка диаграммы Ганта
def Gantt(JOBS, SCHEDULE={}, ax1=None, ax2=None):
    bw = 0.3

    idx = 0
    for j in sorted(JOBS.keys()):
        x = JOBS[j]['release']
        y = JOBS[j]['due']
        ax1.fill_between([x,y],[idx-bw,idx-bw],[idx+bw,idx+bw], color='cyan', alpha=0.6)
        if j in SCHEDULE.keys():
            x = SCHEDULE[j]['start']
            y = SCHEDULE[j]['finish']
            ax1.fill_between([x,y],[idx-bw,idx-bw],[idx+bw,idx+bw], color='red', alpha=0.5)
            ax1.plot([x,y,y,x,x], [idx-bw,idx-bw,idx+bw,idx+bw,idx-bw],color='k')
            ax1.text((SCHEDULE[j]['start'] + SCHEDULE[j]['finish'])/2.0,idx,
                'Job ' + j, color='white', weight='bold',
                horizontalalignment='center', verticalalignment='center')
        idx += 1

    ax1.set_ylim(-0.5, idx-0.5)
    ax1.set_title('Job Schedule')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Jobs')
    ax1.set_yticks(range(len(JOBS)), JOBS.keys())
    ax1.grid()
    xlim = ax1.get_xlim()

    if SCHEDULE:
        for j in SCHEDULE.keys():
            if 'machine' not in SCHEDULE[j].keys():
                SCHEDULE[j]['machine'] = 1
        MACHINES = sorted(set([SCHEDULE[j]['machine'] for j in SCHEDULE.keys()]))

        for j in sorted(SCHEDULE.keys()):
            idx = MACHINES.index(SCHEDULE[j]['machine'])
            x = SCHEDULE[j]['start']
            y = SCHEDULE[j]['finish']
            ax2.fill_between([x,y],[idx-bw,idx-bw],[idx+bw,idx+bw], color='red', alpha=0.5)
            ax2.plot([x,y,y,x,x], [idx-bw,idx-bw,idx+bw,idx+bw,idx-bw],color='k')
            ax2.text((SCHEDULE[j]['start'] + SCHEDULE[j]['finish'])/2.0,idx,
                'Job ' + j, color='white', weight='bold',
                horizontalalignment='center', verticalalignment='center')
        ax2.set_xlim(xlim)
        ax2.set_ylim(-0.5, len(MACHINES)-0.5)
        ax2.set_title('Machine Schedule')
        ax2.set_yticks(range(len(MACHINES)), MACHINES)
        ax2.set_ylabel('Machines')
        ax2.grid()

# Подсчет метрик для решенной задачи
def Kpi(JOBS, SCHEDULE):
    KPI = {}
    KPI['Makespan'] = max(SCHEDULE[job]['finish'] for job in SCHEDULE)
    KPI['Max Pastdue'] = max(max(0, SCHEDULE[job]['finish'] - JOBS[job]['due']) for job in SCHEDULE)
    KPI['AE due'] = max(abs(SCHEDULE[job]['finish'] - JOBS[job]['due']) for job in SCHEDULE)
    KPI['Sum of Pastdue'] = sum(max(0, SCHEDULE[job]['finish'] - JOBS[job]['due']) for job in SCHEDULE)
    KPI['Number Pastdue'] = sum(SCHEDULE[job]['finish'] > JOBS[job]['due'] for job in SCHEDULE)
    KPI['Number on Time'] = sum(SCHEDULE[job]['finish'] <= JOBS[job]['due'] for job in SCHEDULE)
    KPI['Fraction on Time'] = KPI['Number on Time']/len(SCHEDULE)
    return KPI

## Реализация в Pyomo

In [115]:
def ObjectiveF(JOBS, MACHINES, z):
    # матрица длительности i работы в j ой группе (на j машине)
    TMAX = [[0 for _ in range(len(MACHINES))] for _ in range(len(JOBS))]
    # матрица веса i работы в jой группе (на j машине)
    ZMAX = [[0 for _ in range(len(MACHINES))] for _ in range(len(JOBS))]
    # суммарная длительность работ в j ой группе (на j машине)
    MAX = [0] * len(MACHINES)
    # суммарный вес работ в j ой группе (на j машине)
    WMAX = [0] * len(MACHINES)
    MAX_adapt = [0] * len(MACHINES)
    string = list(JOBS.keys())

    for j in range(len(MACHINES)):
        for i in range(len(JOBS)):
            TMAX[i][j] = JOBS[string[i]]['duration'] * z[string[i], MACHINES[j]]
            ZMAX[i][j] = JOBS[string[i]]['weight'] * z[string[i], MACHINES[j]]
            MAX[j] += TMAX[i][j]
            WMAX[j] += ZMAX[i][j]

    MAX_adapt[0] = MAX[0] + 1
    MAX_adapt[1] = MAX[0] + MAX[1] + 2
    MAX_adapt[2] = MAX[0] + MAX[1] + MAX[2] + 3
#     MAX_adapt[3] = MAX[0] + MAX[1] + MAX[2] + MAX[3] + 4
#     MAX_adapt[4] = MAX[0] + MAX[1] + MAX[2] + MAX[3] + MAX[4] + 5
#     MAX_adapt[5] = MAX[0] + MAX[1] + MAX[2] + MAX[3] + MAX[4] + MAX[5] + 6

    objective = 0
    for j in range(len(MACHINES)):
        objective += MAX_adapt[j] * WMAX[j]

    return objective

def z_init(m, j, mach):
    if mach == "G0":
        return 1

    return 0

#Создание метода решения
def ScheduleMachines(JOBS, MACHINES, TAU):

    m = ConcreteModel()

    m.J = Set(initialize=JOBS.keys())
    m.M = Set(initialize=MACHINES)
    m.PAIRS = Set(initialize = m.J * m.J, dimen=2, filter=lambda m, j, k : j < k)
    m.MPAIRS = Set(initialize = m.M * m.M, dimen=2, filter=lambda m, j, k : j < k)
    
    BigM = 6 + (max([JOBS[j]['release'] for j in m.J]) + sum([JOBS[j]['duration'] for j in m.J]))
    
    m.start = Var(m.J, bounds=(0, 1000))
    m.z = Var(m.J, m.M, domain=Binary, initialize=z_init)
    m.y = Var(m.PAIRS, domain=Binary)
    
    m.null_groups = Var(m.M, domain=Binary)
    
    m.OBJ = Objective(expr = ObjectiveF(JOBS, MACHINES, m.z), sense = minimize)

    m.c1 = Constraint(m.J, rule=lambda m, j:
            m.start[j] >= JOBS[j]['release'])
    m.c2 = Constraint(m.J, rule=lambda m, j:
            m.start[j] + JOBS[j]['duration'] <= TAU * len(m.M) + sum(JOBS[j]['duration'] for j in m.J))
    m.c3 = Constraint(m.J, rule=lambda m, j:
            sum(m.z[j,mach] for mach in m.M) == 1)
    
    m.c4 = Constraint(m.M, rule=lambda m, mach:
            sum(m.z[j,mach] for j in m.J) <= m.null_groups[mach])
    
    m.c5 = Constraint(m.MPAIRS, rule=lambda m, j, k:
            m.null_groups[j] >= m.null_groups[k])
    
    m.d1 = Constraint(m.PAIRS, rule = lambda m, j, k:
            m.start[j] + JOBS[j]['duration'] <= m.start[k] + BigM*(m.y[j,k]))
    m.d2 = Constraint(m.PAIRS, rule = lambda m, j, k:
            m.start[k] + JOBS[k]['duration'] <= m.start[j] + BigM*(1-m.y[j,k]))

    m.display()
    solver = SolverFactory("bonmin", executable=r"..\solvers\ampl.mswin64\bonmin.exe", tee=True)
    solver.solve(m)
    
    print(m.null_groups)
    
    SCHEDULE = {}
    for j in m.J:
        SCHEDULE[j] = {
            'start': m.start[j](),
            'finish': m.start[j]() + JOBS[j]['duration'],
            'machine': [mach for mach in MACHINES if m.z[j,mach]()][0]
        }

    return SCHEDULE, m

## Жадный алгоритм

In [116]:
def Test(name, verbose = False):
    DATA = ReadJson(name=name)

    #Задание словаря работ
    JOBS = DATA["JOBS"]
    
    MACHINES = [f"G{i}" for i in range(len(JOBS)-3)]
    TAU = DATA["TAU"]

    SCHEDULE1, MODEL = ScheduleMachines(JOBS.copy(), MACHINES, TAU)
#     SCHEDULE2 = Solve(JOBS.copy(), MACHINES)
    
#     if not verbose:
    fig, ax = plt.subplots(3, 2, figsize=(15,18))
    fig.suptitle(f"{name}")
    ax[0][0].get_xaxis().set_visible(False)
    ax[0][0].get_yaxis().set_visible(False)
    ax[0][0].set_axis_off()
    ax[0][0].text(0.5, 0.5, f"Best: {Kpi(JOBS, SCHEDULE1)['AE due']}\n"
                 f"Greedy: {Kpi(JOBS, SCHEDULE1)['AE due']}")

    Gantt(JOBS, SCHEDULE1, ax[1][0], ax[2][0])
    Gantt(JOBS, SCHEDULE1, ax[1][1], ax[2][1])

    return SCHEDULE1, Kpi(JOBS, SCHEDULE1)['AE due'], Kpi(JOBS, SCHEDULE1)['AE due']

In [117]:
# WriteJson(script.create_task(6, 3, 8, 0, 7))
Test("task1.json")

Model unknown

  Variables:
    start : Size=6, Index=J
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         J1 :     0 :  None :  1000 : False :  True :  Reals
         J2 :     0 :  None :  1000 : False :  True :  Reals
         J3 :     0 :  None :  1000 : False :  True :  Reals
         J4 :     0 :  None :  1000 : False :  True :  Reals
         J5 :     0 :  None :  1000 : False :  True :  Reals
         J6 :     0 :  None :  1000 : False :  True :  Reals
    z : Size=18, Index=z_index
        Key          : Lower : Value : Upper : Fixed : Stale : Domain
        ('J1', 'G0') :     0 :     1 :     1 : False : False : Binary
        ('J1', 'G1') :     0 :     0 :     1 : False : False : Binary
        ('J1', 'G2') :     0 :     0 :     1 : False : False : Binary
        ('J2', 'G0') :     0 :     1 :     1 : False : False : Binary
        ('J2', 'G1') :     0 :     0 :     1 : False : False : Binary
        ('J2', 'G2') :     0 :     0 :     1 : False : False : Bin

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
Test("task2.json")